Currently using code taken from
https://gist.github.com/parulnith/7f8c174e6ac099e86f0495d3d9a4c01e#file-music_genre_classification-ipynb
for exploration of possible solutions 

# Music genre classification notebook

## Importing Libraries

In [1]:
# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


## Extracting music and features

### Dataset

We use [GTZAN genre collection](http://marsyasweb.appspot.com/download/data_sets/) dataset for classification. 
<br>
<br>
The dataset consists of 10 genres i.e
 * Blues
 * Classical
 * Country
 * Disco
 * Hiphop
 * Jazz
 * Metal
 * Pop
 * Reggae
 * Rock
 
Each genre contains 100 songs. Total dataset: 1000 songs

## Extracting the Spectrogram for every Audio

In [2]:
cmap = plt.get_cmap('inferno')

plt.figure(figsize=(10,10))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'../../audio/testfiles/GTZAN/genres/{g}'):
        songname = f'../../audio/testfiles/GTZAN/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()
 

<Figure size 720x720 with 0 Axes>

All the audio files get converted into their respective spectrograms .WE can noe easily extract features from them.

## Extracting features from Spectrogram


We will extract

* Mel-frequency cepstral coefficients (MFCC)(20 in number)
* Spectral Centroid,
* Zero Crossing Rate
* Chroma Frequencies
* Spectral Roll-off.

In [3]:
header = 'filename chroma_stft spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

## Writing data to csv file

We write the data to a csv file 

In [4]:
file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
filepath = '../../audio/testfiles/GTZAN/genres/'
for g in genres:
    for filename in os.listdir(f'{filepath}/{g}'):
        songname = f'{filepath}/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        #rmse = librosa.feature.rmse(y=y, S=None, frame_length=2048, hop_length=512, center=True, pad_mode='reflect')
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        #to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

# Analysing the Data in Pandas

In [5]:
data = pd.read_csv('data.csv')
data.head()

,filename,chroma_stft,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00085.wav,0.315363,1312.308199,1673.915613,2638.117381,0.059416,-180.123596,131.420259,0.566188,43.152929,...,2.718054,0.772589,1.997588,-6.093858,3.484042,-8.341479,3.204648,-0.926944,-2.243686,blues
1,blues.00051.wav,0.393756,1977.172377,1927.803692,3942.834492,0.106627,-55.579243,114.935848,-37.052831,64.896513,...,12.782317,-16.528681,3.793788,-7.890870,8.477609,-4.065210,3.207441,-5.178250,-1.279524,blues
2,blues.00038.wav,0.265883,1513.422107,2140.606779,3449.679140,0.044378,-192.667187,111.190774,21.372723,26.398341,...,-5.044727,-17.631820,-7.921687,-15.152404,-12.342546,-17.227765,-7.771728,-13.033961,-17.945750,blues
3,blues.00069.wav,0.291884,2371.099278,2209.699346,5004.111407,0.125141,-121.408845,96.106080,-19.613255,48.011654,...,6.793616,-19.333882,-0.114447,-12.250433,-4.927438,-10.409581,-2.049566,-1.768211,-3.962159,blues
4,blues.00049.wav,0.277484,1318.656822,1904.761177,3046.681577,0.039973,-255.965774,110.823046,21.505215,67.571233,...,-2.442929,-7.130865,-11.163917,-12.609408,2.742009,-8.948849,-7.940728,-2.344157,-0.705606,blues


In [6]:
data.shape

(1000, 27)

In [7]:
# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)

## Encoding the Labels

In [8]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

## Scaling the Feature columns

In [9]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

## Dividing data into training and Testing set

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [11]:
len(y_train)

800

In [12]:
len(y_test)

200

In [13]:
X_train[10]

array([ 1.0574891 ,  2.58660906,  2.24995798,  2.42092734,  2.56082714,
        0.91680989, -1.65333673,  1.03200834, -2.04530466,  1.17908317,
       -0.4646132 ,  1.96990119, -0.88415511,  1.98758265, -0.09116011,
        0.78221431, -0.40849949,  2.15487121,  1.02655532,  1.44677634,
        0.24468331,  0.67515747,  0.39590775,  1.22952235,  1.19075062])

# Classification with Keras

## Building our Network

In [14]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [15]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
history = model.fit(X_train,
                    y_train,
                    epochs=200,
                    batch_size=128)
                   

Instructions for updating:
Use tf.cast instead.
Epoch 1/200
800/800 [==============================] - 0s 418us/step - loss: 2.2025 - acc: 0.2025
Epoch 2/200
800/800 [==============================] - 0s 31us/step - loss: 1.9097 - acc: 0.3588
Epoch 3/200
800/800 [==============================] - 0s 30us/step - loss: 1.6857 - acc: 0.4025
Epoch 4/200
800/800 [==============================] - 0s 29us/step - loss: 1.4918 - acc: 0.4863
Epoch 5/200
800/800 [==============================] - 0s 30us/step - loss: 1.3394 - acc: 0.5325
Epoch 6/200
800/800 [==============================] - 0s 28us/step - loss: 1.2260 - acc: 0.5862
Epoch 7/200
800/800 [==============================] - 0s 31us/step - loss: 1.1501 - acc: 0.6062
Epoch 8/200
800/800 [==============================] - 0s 31us/step - loss: 1.0687 - acc: 0.6225
Epoch 9/200
800/800 [==============================] - 0s 29us/step - loss: 0.9914 - acc: 0.6713
Epoch 10/200
800/800 [==============================] - 0s 30us/step - loss: 0

800/800 [==============================] - 0s 31us/step - loss: 0.0403 - acc: 0.9988
Epoch 82/200
800/800 [==============================] - 0s 28us/step - loss: 0.0332 - acc: 0.9975
Epoch 83/200
800/800 [==============================] - 0s 31us/step - loss: 0.0323 - acc: 0.9975
Epoch 84/200
800/800 [==============================] - 0s 32us/step - loss: 0.0350 - acc: 0.9962
Epoch 85/200
800/800 [==============================] - 0s 31us/step - loss: 0.0312 - acc: 0.9962
Epoch 86/200
800/800 [==============================] - 0s 30us/step - loss: 0.0254 - acc: 0.9975
Epoch 87/200
800/800 [==============================] - 0s 30us/step - loss: 0.0235 - acc: 0.9975
Epoch 88/200
800/800 [==============================] - 0s 31us/step - loss: 0.0236 - acc: 0.9975
Epoch 89/200
800/800 [==============================] - 0s 30us/step - loss: 0.0224 - acc: 0.9988
Epoch 90/200
800/800 [==============================] - 0s 30us/step - loss: 0.0219 - acc: 0.9975
Epoch 91/200
800/800 [===========

800/800 [==============================] - 0s 29us/step - loss: 0.0065 - acc: 0.9988
Epoch 165/200
800/800 [==============================] - 0s 31us/step - loss: 0.0068 - acc: 0.9988
Epoch 166/200
800/800 [==============================] - 0s 32us/step - loss: 0.0193 - acc: 0.9975
Epoch 167/200
800/800 [==============================] - 0s 31us/step - loss: 0.0118 - acc: 0.9950
Epoch 168/200
800/800 [==============================] - 0s 29us/step - loss: 0.0100 - acc: 0.9975
Epoch 169/200
800/800 [==============================] - 0s 30us/step - loss: 0.0077 - acc: 0.9988
Epoch 170/200
800/800 [==============================] - 0s 30us/step - loss: 0.0060 - acc: 0.9988
Epoch 171/200
800/800 [==============================] - 0s 32us/step - loss: 0.0055 - acc: 0.9988
Epoch 172/200
800/800 [==============================] - 0s 29us/step - loss: 0.0063 - acc: 0.9988
Epoch 173/200
800/800 [==============================] - 0s 29us/step - loss: 0.0053 - acc: 0.9975
Epoch 174/200
800/800 [=

In [17]:
test_loss, test_acc = model.evaluate(X_test,y_test)

200/200 [==============================] - 0s 195us/step


In [18]:
print('test_acc: ',test_acc)

test_acc:  0.635


Tes accuracy is less than training dataa accuracy. This hints at Overfitting

## Validating our approach
Let's set apart 200 samples in our training data to use as a validation set:

In [19]:
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

Now let's train our network for 20 epochs:

In [20]:
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=200,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)

Train on 600 samples, validate on 200 samples
Epoch 1/200
600/600 [==============================] - 0s 593us/step - loss: 2.2892 - acc: 0.1000 - val_loss: 2.1609 - val_acc: 0.2950
Epoch 2/200
600/600 [==============================] - 0s 45us/step - loss: 2.1185 - acc: 0.3350 - val_loss: 2.0363 - val_acc: 0.3200
Epoch 3/200
600/600 [==============================] - 0s 44us/step - loss: 1.9659 - acc: 0.3733 - val_loss: 1.9150 - val_acc: 0.3500
Epoch 4/200
600/600 [==============================] - 0s 42us/step - loss: 1.8125 - acc: 0.3833 - val_loss: 1.7963 - val_acc: 0.3750
Epoch 5/200
600/600 [==============================] - 0s 42us/step - loss: 1.6777 - acc: 0.3967 - val_loss: 1.6781 - val_acc: 0.4350
Epoch 6/200
600/600 [==============================] - 0s 43us/step - loss: 1.5606 - acc: 0.4483 - val_loss: 1.5703 - val_acc: 0.4550
Epoch 7/200
600/600 [==============================] - 0s 41us/step - loss: 1.4577 - acc: 0.4867 - val_loss: 1.4844 - val_acc: 0.4650
Epoch 8/200
600

600/600 [==============================] - 0s 44us/step - loss: 0.1463 - acc: 0.9783 - val_loss: 1.0022 - val_acc: 0.6850
Epoch 62/200
600/600 [==============================] - 0s 44us/step - loss: 0.1454 - acc: 0.9767 - val_loss: 1.0014 - val_acc: 0.6900
Epoch 63/200
600/600 [==============================] - 0s 45us/step - loss: 0.1398 - acc: 0.9783 - val_loss: 1.0032 - val_acc: 0.6600
Epoch 64/200
600/600 [==============================] - 0s 44us/step - loss: 0.1256 - acc: 0.9800 - val_loss: 1.0300 - val_acc: 0.6800
Epoch 65/200
600/600 [==============================] - 0s 45us/step - loss: 0.1328 - acc: 0.9733 - val_loss: 0.9986 - val_acc: 0.6800
Epoch 66/200
600/600 [==============================] - 0s 46us/step - loss: 0.1168 - acc: 0.9800 - val_loss: 0.9939 - val_acc: 0.7000
Epoch 67/200
600/600 [==============================] - 0s 42us/step - loss: 0.1157 - acc: 0.9817 - val_loss: 1.0161 - val_acc: 0.6850
Epoch 68/200
600/600 [==============================] - 0s 44us/step

600/600 [==============================] - 0s 46us/step - loss: 0.0216 - acc: 0.9983 - val_loss: 1.3515 - val_acc: 0.6750
Epoch 122/200
600/600 [==============================] - 0s 48us/step - loss: 0.0192 - acc: 0.9967 - val_loss: 1.3489 - val_acc: 0.6950
Epoch 123/200
600/600 [==============================] - 0s 43us/step - loss: 0.0214 - acc: 0.9967 - val_loss: 1.3331 - val_acc: 0.7000
Epoch 124/200
600/600 [==============================] - 0s 47us/step - loss: 0.0192 - acc: 0.9967 - val_loss: 1.3187 - val_acc: 0.7000
Epoch 125/200
600/600 [==============================] - 0s 45us/step - loss: 0.0164 - acc: 0.9983 - val_loss: 1.3275 - val_acc: 0.6850
Epoch 126/200
600/600 [==============================] - 0s 45us/step - loss: 0.0163 - acc: 0.9983 - val_loss: 1.3459 - val_acc: 0.6800
Epoch 127/200
600/600 [==============================] - 0s 43us/step - loss: 0.0172 - acc: 0.9983 - val_loss: 1.3537 - val_acc: 0.6750
Epoch 128/200
600/600 [==============================] - 0s 50

600/600 [==============================] - 0s 43us/step - loss: 0.0074 - acc: 0.9983 - val_loss: 1.5402 - val_acc: 0.6750
Epoch 182/200
600/600 [==============================] - 0s 43us/step - loss: 0.0067 - acc: 0.9983 - val_loss: 1.5562 - val_acc: 0.6800
Epoch 183/200
600/600 [==============================] - 0s 46us/step - loss: 0.0066 - acc: 0.9983 - val_loss: 1.5704 - val_acc: 0.6750
Epoch 184/200
600/600 [==============================] - 0s 48us/step - loss: 0.0078 - acc: 0.9983 - val_loss: 1.5801 - val_acc: 0.6800
Epoch 185/200
600/600 [==============================] - 0s 47us/step - loss: 0.0097 - acc: 0.9983 - val_loss: 1.5801 - val_acc: 0.6750
Epoch 186/200
600/600 [==============================] - 0s 47us/step - loss: 0.0095 - acc: 0.9983 - val_loss: 1.5796 - val_acc: 0.6850
Epoch 187/200
600/600 [==============================] - 0s 46us/step - loss: 0.0081 - acc: 0.9983 - val_loss: 1.5756 - val_acc: 0.6700
Epoch 188/200
600/600 [==============================] - 0s 44

In [21]:
results

[1.911709065437317, 0.64]

## Predictions on Test Data

In [22]:
predictions = model.predict(X_test)

In [23]:
predictions[0].shape

(10,)

In [24]:
np.sum(predictions[0])

0.9999999

In [25]:
np.argmax(predictions[0])

1